In [1]:
from src.db.models import DbInstance
import json
from src.schemas.catalog.instance import BfInstance
from src.db.init_db import session
from src.function.catalog.instance.graphInstance import MakeGraphInstance

In [2]:
with open('test.json', encoding="utf-8") as file:
    reader = file.read()
    obj = json.loads(reader)

request = BfInstance(**obj)
uri = f'https://bibliokeia.com/catalog/instances/{request.identifiersLocal}'
work_id = request.instanceOf.uri.split("/")[-1]

In [4]:
graph = MakeGraphInstance(request)
print(graph)


PREFIX bki: <https://bibliokeia.com/resources/instance/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
        
    INSERT DATA {
        GRAPH bki:6
        {
                bki:6 a bf:Instance, 
                    bf:Print;
                bf:adminMetadata [ a bf:AdminMetadata ;
                bf:assigner <http://

In [5]:
from src.function.catalog.solr.docInstance import DocInstance

In [6]:
responseSolr = DocInstance(request)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":4148\n  }\n}'

In [7]:
request

BfInstance(adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', changeDate=None, creationDate=datetime.date(2023, 12, 10), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/local', generationProcess='BiblioKeia v.1', generationDate='2023-12-10T19:59:51', status=Status(value='n', label='novo')), identifiersLocal='1', type='Print', media=Element(label='NÃ£o mediado', lang=None, uri='http://id.loc.gov/vocabulary/mediaTypes/n', type=None), issuance=Element(label='Monografia', lang=None, uri='http://id.loc.gov/vocabulary/issuance/mono', type=None), carrier=Element(label='Volume', lang=None, uri='http://id.loc.gov/vocabulary/carriers/nc', type=None), title=Title(type='bf:Title', mainTitle='A maÃ§Ã£ no escuro', subtitle='', label=None), publication=Publication(agent='Companhia das letras', 